In [2]:
# Import dependencies
import requests
import json
import pandas as pd
import csv
from io import StringIO

In [3]:
# Define the base URL for querying the Earthquake layer
query_url = "https://maps.texnet.beg.utexas.edu/arcgis/rest/services/catalog/catalog_all_flat/MapServer/0/query"

# Define the parameters for the query
params = {
    "where": "1=1",  # Fetches all earthquake records
    "outFields": "Magnitude,Latitude,Longitude,Event_Date,GeographicLocation",  # Specify the fields of interest
    "returnGeometry": True,  #might be redundant with f=geojson
    "outSR": "4326",  # Output Spatial Reference - WGS 84
    "f": "geojson"  # Request output format as GeoJSON
}

# Make the request to the ArcGIS REST API
response = requests.get(query_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # The response should already be in GeoJSON format
    geojson_data = response.json()

    # Save the GeoJSON data to a file
    with open('earthquake_data.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)

    print("Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}, Message: {response.text}")


Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.


In [4]:
# Make the request to the ArcGIS REST API
response = requests.get(query_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # The response should already be in GeoJSON format
    geojson_data = response.json()

    # Save the GeoJSON data to a file
    with open('earthquake_data.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)

    print("Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}, Message: {response.text}")

Earthquake GeoJSON data has been saved to 'earthquake_data.geojson'.


In [5]:
geojson_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-97.94009999999997, 28.955400000000054]},
   'properties': {'Magnitude': 2.508384648,
    'Latitude': 28.9553833,
    'Longitude': -97.94014246,
    'Event_Date': 1482252218000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-98.00339999999994, 32.71090000000004]},
   'properties': {'Magnitude': 1.34568779,
    'Latitude': 32.71087646,
    'Longitude': -98.00340606,
    'Event_Date': 1483172174000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-96.93909999999994, 32.85460000000006]},
   'properties': {'Magnitude': 2.046221503,
    'Latitude': 32.85461426,
    'Longitude': -96.93907203,
    'Event_Date': 1483756689000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-97.30779999999999, 32.69350000000003]},
   'properties': {'Magnitude': 1.901215261,
    'Latitude': 32.69348145,
    '

In [5]:
# Initialize lists to store data
latitude = []
longitude = []
magnitude = []
event_date = []

In [6]:
# Extract data from features
for feature in geojson_data['features']:
    latitude.append(feature['properties']['Latitude'])
    longitude.append(feature['properties']['Longitude'])
    magnitude.append(feature['properties']['Magnitude'])
    event_date.append(feature['properties']['Event_Date'])

In [7]:
# Create DataFrame
df_earthquakes = pd.DataFrame({
    'Latitude': latitude,
    'Longitude': longitude,
    'Magnitude': magnitude,
    'Event_Date': event_date
})

In [8]:
# Display the DataFrame
df_earthquakes.head()

,Latitude,Longitude,Magnitude,Event_Date
0,28.955383,-97.940142,2.508385,1482252218000
1,32.710876,-98.003406,1.345688,1483172174000
2,32.854614,-96.939072,2.046222,1483756689000
3,32.693481,-97.307784,1.901215,1483835854000
4,32.670593,-97.246689,2.197399,1484027207000


In [9]:
# Convert the event_date column values into calendar dates and times
df_earthquakes['Event_Date'] = pd.to_datetime(df_earthquakes['Event_Date'], unit='ms')

# Display the DataFrame
df_earthquakes.head()

,Latitude,Longitude,Magnitude,Event_Date
0,28.955383,-97.940142,2.508385,2016-12-20 16:43:38
1,32.710876,-98.003406,1.345688,2016-12-31 08:16:14
2,32.854614,-96.939072,2.046222,2017-01-07 02:38:09
3,32.693481,-97.307784,1.901215,2017-01-08 00:37:34
4,32.670593,-97.246689,2.197399,2017-01-10 05:46:47


In [10]:
# Remove times and keep only dates in the 'Event_Date' column
df_earthquakes['Event_Date'] = pd.to_datetime(df_earthquakes['Event_Date']).dt.date

# Display the DataFrame
df_earthquakes.head()

,Latitude,Longitude,Magnitude,Event_Date
0,28.955383,-97.940142,2.508385,2016-12-20
1,32.710876,-98.003406,1.345688,2016-12-31
2,32.854614,-96.939072,2.046222,2017-01-07
3,32.693481,-97.307784,1.901215,2017-01-08
4,32.670593,-97.246689,2.197399,2017-01-10


In [13]:
df_earthquakes['Event_Date'] = pd.to_datetime(df_earthquakes['Event_Date'])
df_earthquakes.dtypes

Latitude             float64
Longitude            float64
Magnitude            float64
Event_Date    datetime64[ns]
dtype: object

In [12]:
# Export the Clean DataFrame as a csv File
df_earthquakes.to_csv('earthquakes.csv', index=False)